# IR - Assignment 2

We will start with 2 groups of documents (from 4 sources):

1. word group
2. lemma group

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
aj_word_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word1/A_J_word.csv?raw=true"
bbc_wrod_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word1/BBC_word.csv?raw=true"
jp_word_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word1/J_P_word.csv?raw=true"
nyt_word_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word1/NYT_word.csv?raw=true"

group_word = {'aj': aj_word_link, 'bbc': bbc_wrod_link, 'jp': jp_word_link, 'nyt': nyt_word_link}

In [3]:
# aj_lemma_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/lemma1/A_J_lemma.csv?raw=true"
# bbc_lemma_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/lemma1/BBC_lemma.csv?raw=true"
# jp_lemma_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/lemma1/J_P_lemma.csv?raw=true"
# nyt_lemma_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/lemma1/NYT_lemma.csv?raw=true"

# group_lemma = {'aj_l': aj_lemma_link, 'bbc_l': bbc_lemma_link, 'jp_l': jp_lemma_link, 'nyt_l': nyt_lemma_link}

In [3]:
def download_data(link):
    import pandas as pd

    df = pd.read_csv(link)
    return df

We will run a test for one journal and then generilize

We will take the df that contains the following columns:

1. `id` - for example `aj_1`.
2. `document` - the text of the document clean in two different ways depending on the group (word, lemma).

We will create a matrix for each journal the matrix will be as follows:

1. a vector representation for each of the document using `doc2vec`

In [4]:
import nltk
import string
import pandas as pd

In [5]:
nltk.download('punkt')  # Download punkt tokenizer if not already downloaded
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    # You can add further cleaning steps here if needed
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]

    # remove all punctuation marks
    tokens = [word.translate(str.maketrans('', '', string.punctuation)) for word in tokens]

    return tokens

In [7]:
# import the doc2vec model and for each line in our df create a vector of the document
# and then assembele them into a matrix

from gensim.models.doc2vec import Doc2Vec, TaggedDocument


# apply for each line of the df

# documents_uncleaned = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['tokens_uncleaned'])]

# model_uncleaned = Doc2Vec(documents_uncleaned, vector_size=100, window=5, min_count=1, workers=4)

In [14]:
# matrix_doc2vec = [model_uncleaned.infer_vector(doc) for doc in df['tokens_uncleaned']]

In [15]:
# # transoform matrix into a csv file with name aj_word_2_vec.csv

# import pandas as pd

# df_matrix = pd.DataFrame(matrix)
# df_matrix.to_csv('aj_word_2_vec.csv', index=False)

## Step 2 - generalize

In [8]:
dfs = {name: download_data(link) for name, link in group_word.items()}

In [9]:
# create folder for storing the output

import os

if not os.path.exists('output'):
    os.makedirs('output')

In [21]:
#!rm -rf output

In [24]:
for name, df in dfs.items():
  print(f"processing {name}")
  df['tokens'] = df['document'].apply(preprocess_text)
  documents_uncleaned = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['tokens'])]
  model_uncleaned = Doc2Vec(documents_uncleaned, vector_size=100, window=5, min_count=1, workers=4)

  matrix = [model_uncleaned.infer_vector(doc) for doc in df['tokens']]
  df_matrix = pd.DataFrame(matrix)
  df_matrix.to_csv(f'output/{name}_doc2vec.csv', index=False)
  print(f"output save in output/{name}_doc2vec.csv.csv")

processing aj
output save in output/aj_doc2vec.csv.csv
processing bbc
output save in output/bbc_doc2vec.csv.csv
processing jp
output save in output/jp_doc2vec.csv.csv
processing nyt
output save in output/nyt_doc2vec.csv.csv


# Step 2 - Bert and Bert Sentence vectors

In [10]:
!pip install transformers==4.31.0
!pip install -U sentence-transformers

  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.31.0 which is incompatible.
  Using cached 

In [11]:
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer
import torch

In [12]:
# For BERT
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# For BERT-sentence
bert_sentence_model = SentenceTransformer('bert-base-nli-mean-tokens')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np


def get_bert_embeddings(text):
  inputs = bert_tokenizer(text, padding=True, truncation=True, return_tensors='pt')
  outputs = bert_model(**inputs)
  embeddings = outputs.last_hidden_state[:, 0, :].detach().numpy()
  return embeddings

def get_bert_sentence_embeddings(text):
  embeddings = bert_sentence_model.encode(text)
  return embeddings


In [24]:

for key, df in dfs.items():
    print(f"processing {key}")

    # Get embeddings for column 'document'
    bert_embedding = df['document'].apply(get_bert_embeddings)
    bert_sentence_embeddings = df['document'].apply(get_bert_sentence_embeddings)

    # 1. Stack the embeddings into a single matrix
    bert_embedding_matrix = np.vstack(bert_embedding)
    bert_sentence_embeddings_matrix = np.vstack(bert_sentence_embeddings)

    # 2. Apply PCA on the matrices
    pca = PCA(n_components=100)
    bert_embedding_reduced = pca.fit_transform(bert_embedding_matrix)
    bert_sentence_embeddings_reduced = pca.fit_transform(bert_sentence_embeddings_matrix)

    # 3. Convert to DataFrame and save
    bert_embedding_df = pd.DataFrame(bert_embedding_reduced, columns=[str(i) for i in range(100)])
    bert_embedding_df.to_csv(f'output/{key}_bert_embedding_100d.csv', index=False)

    bert_sentence_embeddings_df = pd.DataFrame(bert_sentence_embeddings_reduced, columns=[str(i) for i in range(100)])
    bert_sentence_embeddings_df.to_csv(f'output/{key}_bert_sentence_embeddings_100d.csv', index=False)

    print(f"output save in output/{key}_bert_embedding_100d.csv and output/{key}_bert_sentence_embeddings_100d.csv")

processing aj
output save in output/aj_bert_embedding_100d.csv and output/aj_bert_sentence_embeddings_100d.csv
processing bbc
output save in output/bbc_bert_embedding_100d.csv and output/bbc_bert_sentence_embeddings_100d.csv
processing jp
output save in output/jp_bert_embedding_100d.csv and output/jp_bert_sentence_embeddings_100d.csv
processing nyt
output save in output/nyt_bert_embedding_100d.csv and output/nyt_bert_sentence_embeddings_100d.csv
